In [1]:
import numpy as np
import time, os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from cellpose import models, core

use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
from cellpose import utils

import random
from skimage import io
import tqdm
import napari
import pandas as pd

import zarr
from dask import array as da

>>> GPU activated? 1


In [2]:
# path to dataset and model
dataset_folder = "/mnt/ampa02_data01/gabacoll/shared/Yuchen/240417_whole_4color_1st_M037-3pb/model_training/crops"
aug_folder = os.path.join(dataset_folder, 'augment')
train_folder = os.path.join(aug_folder,'training')
models_path = os.path.join(train_folder,'models')

models_file = os.listdir(models_path); models_file.sort()
model_path = os.path.join(train_folder,'models',models_file[-1])

model = models.CellposeModel(gpu=use_GPU, pretrained_model=model_path)

In [15]:
# image path
fix_n5_path = '/mnt/ampa02_data01/tmurakami/240425_whole_4color_2nd_M037-3pb/fused/fused.n5' # zarr with pyramid resolution

# create Zarr file object
fix_zarr = zarr.open(store=zarr.N5Store(fix_n5_path), mode='r')
voxel_size = (2.0,1.3,1.3)

corner_positions = [1209,6600,1854]
crop_size = [128,512,512]
segment_chan = 0
reference_chan = 3

# Channel parameters which were used during the training.
Training_channel = 2 # I do not know but the cellpose see the images as KRGB. If the color is green, set it to 2.
Second_training_channel = 1

# load images according to the input parameters.
n5_setups = list(fix_zarr.keys())
img_ref = fix_zarr[n5_setups[reference_chan]]['timepoint0']['s0']
img_ref_ = img_ref[tuple(slice(i,i+j) for i,j in zip(corner_positions, crop_size))]

img = fix_zarr[n5_setups[segment_chan]]['timepoint0']['s0']
img_ = img[tuple(slice(i,i+j) for i,j in zip(corner_positions, crop_size))]

imgs = np.stack([img_ref_,img_])

In [16]:
# theoretically, anisotropy parameter affects the accuracy. However in practice, changing this values to be the exact voxel ratio does not significantly add accuracy. 
# this may be because of the non-isotropic PSF of light-sheet. 
anisotropy = voxel_size[1]/voxel_size[0] 

# with diameter
%time masks, flows, styles  = model.eval(imgs, channels=[Training_channel,Second_training_channel], z_axis=1, diameter=model.diam_mean, do_3D=True, min_size=40, progress=True, anisotropy=anisotropy)
# without diameter
# %time masks, flows, styles = model.eval(imgs, channels=[Training_channel,Second_training_channel], z_axis=1, diameter=None, do_3D=True, min_size=40, progress=True, anisotropy=anisotropy)

CPU times: user 20.8 s, sys: 7.24 s, total: 28.1 s
Wall time: 22 s


In [17]:
viewer = napari.Viewer()
viewer.add_image(imgs, channel_axis=0, name='image01', blending='additive')
viewer.add_labels(masks)

<Labels layer 'masks' at 0x7fdf4ac0e4c0>

In [18]:
#